# zorang2 Code


In [4]:
%load_ext autoreload
%autoreload 2

import numpy as np
import matplotlib.pyplot as plt
import cv2
import tensorflow as tf
from emotion import Emotion
import mediapipe as mp
import csv
import glob
import re
from tqdm import tqdm

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
dirpath = "/media/sldev1/T7/DMS2/"
# filepath = ["/media/sldev1/T7/02. 김두용_1205_test.mp4", '/media/sldev1/T7/03. 주용현_1206_test.mp4']
filepath = glob.glob(dirpath+'*.mp4')
filepath = sorted(filepath ,key=lambda s: int(re.findall(r'\d+', s)[3]))
filepath = filepath[3:]
print(filepath)
emo = Emotion()
mp_face_detection = mp.solutions.face_detection
mp_drawing = mp.solutions.drawing_utils

['/media/sldev1/T7/DMS2/04. 손하늘_1208.mp4', '/media/sldev1/T7/DMS2/05. 이민정_1209.mp4', '/media/sldev1/T7/DMS2/06. 김세희_1213.mp4', '/media/sldev1/T7/DMS2/07. 이선진_1213.mp4', '/media/sldev1/T7/DMS2/08. 정승아_1214.mp4', '/media/sldev1/T7/DMS2/09. 류채원_1214.mp4', '/media/sldev1/T7/DMS2/10. 최서영_1214.mp4', '/media/sldev1/T7/DMS2/11. 하현옥_1214.mp4', '/media/sldev1/T7/DMS2/12. 윤호영_1214.mp4', '/media/sldev1/T7/DMS2/13. 김설양_1215.mp4', '/media/sldev1/T7/DMS2/14. 박세익_1215.mp4', '/media/sldev1/T7/DMS2/15. 김주일_1215.mp4', '/media/sldev1/T7/DMS2/16. 유현지_1215.mp4', '/media/sldev1/T7/DMS2/17. 임우준_1216.mp4', '/media/sldev1/T7/DMS2/18. 송영달_1216.mp4', '/media/sldev1/T7/DMS2/19. 표성민_1216.mp4', '/media/sldev1/T7/DMS2/20. 박동수_1216.mp4', '/media/sldev1/T7/DMS2/21. 유은식_1216.mp4', '/media/sldev1/T7/DMS2/22. 안만석_1219.mp4', '/media/sldev1/T7/DMS2/23. 최시리_1219.mp4', '/media/sldev1/T7/DMS2/24. 서창희_1220.mp4', '/media/sldev1/T7/DMS2/25. 이진성_1220.mp4', '/media/sldev1/T7/DMS2/26. 김영재_1220.mp4', '/media/sldev1/T7/DMS2/27. 김채윤_12

In [ ]:
for i in tqdm(filepath):
    print(i)

    cap = cv2.VideoCapture(i)
    with mp_face_detection.FaceDetection(
        model_selection=0, min_detection_confidence=0.5) as face_detection:

        with open('/home/sldev1/github/TIL/Internship/DMS2_SMU/FER_Result_SL/'+str(i)[22:-4]+'.csv', 'w', newline = '') as file:
            writer = csv.writer(file)
            writer.writerow(["frame", "result"])

            while(cap.isOpened()):
                ret, frame = cap.read()
                rgbframe = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                results = face_detection.process(rgbframe)
                rgbframe.flags.writeable = True
                rgbframe = cv2.cvtColor(rgbframe, cv2.COLOR_RGB2BGR)

                if results.detections:
                    bbox = results.detections[0].location_data.relative_bounding_box
                    top = rgbframe.shape[0]*bbox.ymin
                    bottom = rgbframe.shape[0]*(bbox.xmin+bbox.height)
                    left = rgbframe.shape[1]*bbox.xmin
                    right = rgbframe.shape[1]*(bbox.xmin+bbox.width)

                    pre_emo, one, two = emo.process(frame[int(left)-10:int(right)+10, int(top)-10:int(bottom)+10])
                    # 0: neutral, 1: joy, 2: sadness, 3: surprise, 4: fear, 5: disgust, 6: anger, 7: contemp
                    cv2.putText(frame, f"Emotion : {pre_emo}", (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 200, 0), 2)
                    cv2.putText(frame, f"Frame : {cap.get(1)}", (10, 90), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 200), 2)
                    cv2.putText(frame, f"One : {one}", (10, 120), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 200, 0), 2)
                    cv2.putText(frame, f"two : {two}", (10, 150), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 200, 0), 2)


                    cv2.rectangle(frame, (int(left), int(top)), (int(right), int(bottom)), (255, 255, 0), 3)

                cv2.imshow("frame", frame)

                if cv2.waitKey(1) == ord('q'):
                    break

                with open('/home/sldev1/github/TIL/Internship/DMS2_SMU/FER_Result_SL/'+str(i)[22:-4]+'.csv', 'a', newline = '') as file:
                    writer = csv.writer(file)
                    writer.writerow([cap.get(1), pre_emo])
                    file.close()
                
                if cap.get(1) == cap.get(cv2.CAP_PROP_FRAME_COUNT):
                    break
            cap.release()
            cv2.destroyAllWindows()


# 1258 frame에서 멈추는 중

  0%|                                                                                          | 0/47 [00:00<?, ?it/s]INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


/media/sldev1/T7/DMS2/04. 손하늘_1208.mp4


  2%|█▋                                                                           | 1/47 [59:18<45:27:51, 3558.09s/it]

/media/sldev1/T7/DMS2/05. 이민정_1209.mp4


  4%|███▏                                                                       | 2/47 [1:48:41<40:06:15, 3208.33s/it]

...

/media/sldev1/T7/DMS2/39. 이점석_1223.mp4


 77%|████████████████████████████████████████████████████████▋                 | 36/47 [29:50:36<9:03:34, 2964.92s/it]

/media/sldev1/T7/DMS2/40. 송효석_1223.mp4


 79%|██████████████████████████████████████████████████████████▎               | 37/47 [30:53:49<8:55:33, 3213.33s/it]

/media/sldev1/T7/DMS2/41. 최태진_1223.mp4


## 01/13(금) 오류

In [ ]:
  6%|████▌                                                                      | 3/50 [2:52:17<45:43:23, 3502.20s/it]
/media/sldev1/T7/DMS2/04. 손하늘_1208.mp4
[h264 @ 0x559e98b11500] Invalid NAL unit size (7082 > 1541).
[h264 @ 0x559e98b11500] Error splitting the input into NAL units.
  6%|████▌                                                                      | 3/50 [2:54:15<45:30:00, 3485.13s/it]
---------------------------------------------------------------------------
error                                     Traceback (most recent call last)
/tmp/ipykernel_17891/1244803589.py in <module>
     12             while(cap.isOpened()):
     13                 ret, frame = cap.read()
---> 14                 rgbframe = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
     15                 results = face_detection.process(rgbframe)
     16                 rgbframe.flags.writeable = True

error: OpenCV(4.5.5) /io/opencv/modules/imgproc/src/color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cvtColor'
                
# Frame 2566에서 에러뜸.
# 다시 하니까 또 됨. 머노?